# Data Preprocessing and Feature Engineering

## Libraries

In [ ]:
%load_ext autoreload
%autoreload 2
# Import Libraries
import sqlalchemy
import pandas as pd
import missingno as msno 
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import numpy as np
import matplotlib.ticker as mtick
from CustomLibs.CustomFunctions import plot_corr_heatmap, value_to_float, fig_indexes, sqlcol,what_pct_train
from config import Config
from sklearn.metrics import mean_squared_error,root_mean_squared_error,mean_absolute_error,r2_score
from CustomLibs.CustomTransformers import SpikeRemover, DailyMeanImputer, filtered_transformer
from sklearn.pipeline import Pipeline,make_pipeline
# from imblearn.pipeline import Pipeline 
# from imblearn          import FunctionSampler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder, SplineTransformer, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import TimeSeriesSplit, KFold
import scipy.stats

#import csv



from CustomLibs.MultiPipe import MultiPipe


# pds.CV=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=4, test_size=30)

date_val_end=Config.TEST_DATE_CUTOFF
date_test_start=pd.to_datetime(date_val_end) + pd.DateOffset(days=1)

engine = sqlalchemy.create_engine(Config.CONN_STR)


## Import data and create featureset dataframes

In [ ]:
with engine.connect() as conn:
    # Import leading variables - attributes which are known ahead of time
    with open('.\SQL Files\LoadAllRawFeatures.sql', 'r') as query:
        df_all = pd.read_sql_query(query.read(),conn)
    with open('.\SQL Files\LoadLaggedLabelFeatures.sql', 'r') as query:
        df_ts = pd.read_sql_query(query.read(),conn)
    # df_ts = pd.read_sql_table('Moving_Averages_By_Day', conn,schema='Silver')
    
    df_ts.drop(columns='Pct_On_Site',inplace=True)
    with open('.\SQL Files\LoadMonthlyWasteFeatures.sql', 'r') as query:
        df_waste = pd.read_sql_query(query.read(),conn)
        df_waste['EOMonth']=pd.to_datetime(df_waste['EOMonth'])



# print some details
print(f'df_all, loaded {len(df_all.columns)} columns and {len(df_all)} rows')
print(f'df_ts, loaded {len(df_ts.columns)} columns and {len(df_ts)} rows')

not_enough_data=[]
for columnName, columnData in df_all.items():
    if columnData.count() / len(columnData) < 0.8:
        not_enough_data.append(columnName)

df_all.drop(columns=not_enough_data,inplace=True)

df_coldheat=df_all[['Heat_Consumption_ori','Heat_Consumption', 'Cold_Consumption_ori', 'Cold_Consumption']]
df_electric=df_all[['Day_Electric_KWh_ori','Day_Electric_KWh', 'Night_Electric_KWh_ori', 'Night_Electric_KWh']]
df_vpn_webex=df_all[['VPN_cnxn_ori','VPN_cnxn',
                     'Meeting_Cnxns_ori','Meeting_Cnxns'
                    #  'Webex_Total_Participants_ori','Webex_Total_Participants',
                    #  'Webex_Maximum_Concurrent_Meetings_ori','Webex_Maximum_Concurrent_Meetings',
                     ]]
df_water=df_all[['Water_Consumption_ori','Water_Consumption']]

# ori_updated_list=['Heat_Consumption_ori','Heat_Consumption', 'Cold_Consumption_ori', 'Cold_Consumption']+['Day_Electric_KWh_ori','Day_Electric_KWh', 'Night_Electric_KWh_ori', 'Night_Electric_KWh']+['VPN_cnxn_ori','VPN_cnxn','Webex_Connections_ori','Webex_Connections']+['Water_Consumption_ori','Water_Consumption']
# df_ori_updated=df_all[ori_updated_list]

drop_list = [x for x in df_all.columns if x.endswith('_ori')]

df_all.drop(columns=drop_list,inplace=True)
# df_all.drop(columns=['Day_Electric_KWh_ori', 'Night_Electric_KWh_ori','Heat_Consumption_ori', 'Cold_Consumption_ori'],inplace=True)

# df_all['Avg_Daily_Biodegradable']=np.where(df_all['Work_Days_Per_Month']>15,df_all['Total_Monthly_Biodegradable']/df_all['Work_Days_Per_Month'],np.NaN)
# df_all['Avg_Daily_Landfill']=np.where(df_all['Work_Days_Per_Month']>15,df_all['Total_Monthly_Landfill']/df_all['Work_Days_Per_Month'],np.NaN)
# df_all['Avg_Daily_Recycable']=np.where(df_all['Work_Days_Per_Month']>15,df_all['Total_Monthly_Recycable']/df_all['Work_Days_Per_Month'],np.NaN)
# df_all['Avg_Daily_Waste']=np.where(df_all['Work_Days_Per_Month']>15,df_all['Total_Monthly_Waste']/df_all['Work_Days_Per_Month'],np.NaN)
# df_all.drop(columns=['Total_Monthly_Biodegradable','Total_Monthly_Landfill','Total_Monthly_Recycable','Total_Monthly_Waste'],inplace=True)

df_all['Car_Parking_Occupancy']=df_all['Car_Parking_Occupancy']/df_all['Car_Parking_Capacity']
df_all['Motorbike_Parking_Occupancy']=df_all['Motorbike_Parking_Occupancy']/df_all['Motorbike_Parking_Capacity']
df_all['Bike_Parking_Occupancy']=df_all['Bike_Parking_Occupancy']/df_all['Bike_Parking_Capacity']

df_all['Pct_On_Site']=df_all['Actual_Desks_Used']/df_all['Total_Staff']

df_discarded=df_all[['Date','Actual_Desks_Used','Total_Staff','Car_Parking_Capacity','Motorbike_Parking_Capacity','Bike_Parking_Capacity']]

with engine.connect() as conn:
    df_discarded.to_sql('Discarded_Features',conn,schema='Silver',if_exists='replace',index=False,dtype=sqlcol(df_discarded))

df_discarded.set_index('Date')

df_all.drop(columns=['Actual_Desks_Used','Total_Staff'],inplace=True)
df_all.drop(columns=['Car_Parking_Capacity','Motorbike_Parking_Capacity','Bike_Parking_Capacity'],inplace=True)


# df_all.set_index('Date',inplace=True)
# df_ts.set_index('Date',inplace=True)

df_all.info()
df_waste.info()

time_field=df_all.columns[0]
leading_features=df_all.columns[1:11].tolist()
lagging_features=df_all.columns[11:32].tolist()
waste_features=df_waste.columns[1:].tolist()
periodic_features=df_all.columns[32:34].tolist()
timeseries_features=df_ts.columns[1:].tolist()
label_field = df_all.columns[-1]


print(f'Dropped with missing > 80%: {not_enough_data}')
print(f'Date Field: {time_field}')
print(f'Label Field: {label_field}')
print(f'{len(leading_features)} Leading Features: {leading_features}')
print(f'{len(lagging_features)} Lagging Features: {lagging_features}')
print(f'{len(waste_features)} Waste Features: {waste_features}')
print(f'{len(periodic_features)} Periodic Features: {periodic_features}')
print(f'{len(timeseries_features)} Timeseries Features: {timeseries_features}')









### Application of noise to df_all

In [ ]:
# with engine.connect() as conn:
    # df_all.to_sql('All_Raw_Features_With_Noise',conn,schema='Silver',if_exists='replace',index=False,dtype=sqlcol(df_all))
    # df_pre_noise.set_index('Date').to_sql('Pre_Noise_Data_Copy',conn,schema='Silver',if_exists='replace',dtype=sqlcol(df_pre_noise))

In [ ]:
if Config.REAPPLY_NOISE:

    lower = 0.0
    upper = 2.0
    mu = 0.95
    sigma = 0.1

    rangen1=scipy.stats.truncnorm((lower-mu)/sigma,(upper-mu)/sigma,loc=mu,scale=sigma)
    to_apply_noise = ['Desks_Booked','Annual_Leave','Flexi_Leave','Total_Leave']

    df_pre_noise=df_all.set_index('Date')[to_apply_noise]
    # print(df_pre_noise['Desks_Booked'])

    for field in to_apply_noise:
        df_all[field]=df_all[field].apply(lambda x: x*rangen1.rvs(1)[0])


    with engine.connect() as conn:
        df_all.to_sql('All_Raw_Features_With_Noise',conn,schema='Silver',if_exists='replace',index=False,dtype=sqlcol(df_all))
        df_pre_noise.to_sql('Pre_Noise_Data_Copy',conn,schema='Silver',if_exists='replace',dtype=sqlcol(df_pre_noise))
        df_access_by_directorate = pd.read_sql_table('Moving_Averages_By_Directorate', conn,schema='Silver')
        df_access_by_directorate['Desks_Booked']=df_access_by_directorate['Desks_Booked'].apply(lambda x: x*rangen1.rvs(1)[0])
        df_access_by_directorate.to_sql('Moving_Averages_By_Directorate',conn,schema='Gold',if_exists='replace',index=False,dtype=sqlcol(df_access_by_directorate))
else:
    with engine.connect() as conn:
        df_all=pd.read_sql_table('All_Raw_Features_With_Noise', conn,schema='Silver')
        df_pre_noise=pd.read_sql_table('Pre_Noise_Data_Copy', conn,schema='Silver').set_index('Date')


## Data Corrections
### Electric Values rationalisation

In [ ]:
fig,axs=plt.subplots(2,2,figsize=(10,4),sharex=True,sharey=True)
dates=df_all['Date']
for i,(columnName,columnData) in enumerate(df_electric.items()):
    zmean=np.mean(columnData.loc[lambda x : x != 0])
    x,y=fig_indexes(2,i)
    axs[x][y].hlines(y=zmean,xmin=min(dates),xmax=max(dates), linewidth=0.5, color='r',linestyles='dashed',label='Mean')
    axs[x][y].plot(dates,columnData,label=columnName)
    axs[x][y].grid(visible=True,which='Major',axis='both')
    if y > 0:
        axs[x][y].axvspan(datetime(2023,2,20), datetime(2023,3,10),color='g',alpha=0.2)
        axs[x][y].axvspan(datetime(2023,10,20), datetime(2023,12,10),color='g',alpha=0.2)
        axs[x][y].axvspan(datetime(2023,12,31), datetime(2024,1,31),color='g',alpha=0.2)
    else:
        axs[x][y].axvspan(datetime(2023,2,20), datetime(2023,3,10),color='r',alpha=0.2)
        axs[x][y].axvspan(datetime(2023,10,20), datetime(2023,12,10),color='r',alpha=0.2)
        axs[x][y].axvspan(datetime(2023,12,31), datetime(2024,1,31),color='r',alpha=0.2)
    if (x,y) == (0,0):
        axs[x][y].set_ylabel('Day Usage (Kwh)', fontsize=10)
    elif (x,y) == (1,0):
        axs[x][y].set_ylabel('Night Usage (Kwh)', fontsize=10)
    if x > 0:
        axs[x][y].tick_params(axis='x', labelrotation=45)
    elif y == 0:
        axs[x][y].set_title('Original',fontsize=12)
    else:
        axs[x][y].set_title('Updated',fontsize=12)
    if Config.MASK_VALUE:
        axs[x][y].set_yticklabels([])
plt.suptitle('Electricity Usage Data Quality',fontsize=14)
plt.tight_layout()

fig.savefig('./Output Files/Images/Pre Processing/electricity_corrections.png',format='png',bbox_inches='tight')





### Cold Heat Values Shift

In [ ]:
fig,axs=plt.subplots(2,2,figsize=(10,4),sharex=True,sharey=True)
dates=df_all['Date']

df_coldheat = SpikeRemover(cutvalue=5,cutmode='zthresh').set_output(transform='pandas').fit_transform(df_coldheat)

for i,(columnName,columnData) in enumerate(df_coldheat.items()):
    zmean=np.mean(columnData.loc[lambda x : x != 0])
    x,y=fig_indexes(2,i)
    axs[x][y].hlines(y=zmean,xmin=min(dates),xmax=max(dates), linewidth=0.5, color='r',linestyles='dashed',label='Mean')
    axs[x][y].plot(dates,columnData,label=columnName)
    axs[x][y].grid(visible=True,which='Major',axis='both')
    if y > 0:
        axs[x][y].axvspan(datetime(2022,8,1), datetime(2022,11,30),color='g',alpha=0.2)
        # axs[x][y].axvspan(datetime(2023,10,20), datetime(2023,12,10),color='g',alpha=0.2)
        # axs[x][y].axvspan(datetime(2023,12,31), datetime(2024,1,31),color='g',alpha=0.2)
    else:
        axs[x][y].axvspan(datetime(2022,8,1), datetime(2022,11,30),color='r',alpha=0.2)
        # axs[x][y].axvspan(datetime(2023,10,20), datetime(2023,12,10),color='r',alpha=0.2)
        # axs[x][y].axvspan(datetime(2023,12,31), datetime(2024,1,31),color='r',alpha=0.2)
    if (x,y) == (0,0):
        axs[x][y].set_ylabel('Heat Consumption', fontsize=10)
    elif (x,y) == (1,0):
        axs[x][y].set_ylabel('Cold Consumption', fontsize=10)
    if x > 0:
        axs[x][y].tick_params(axis='x', labelrotation=45)
    elif y == 0:
        axs[x][y].set_title('Original',fontsize=12)
    else:
        axs[x][y].set_title('Updated',fontsize=12)
    axs[x][y].set_yticklabels([])
plt.suptitle('Heat And Cold Consumption Data Quality',fontsize=14)
plt.tight_layout()

fig.savefig('./Output Files/Images/Pre Processing/heatcold_corrections.png',format='png',bbox_inches='tight')





### Water Consumption Shift Correction

In [ ]:
fig,axs=plt.subplots(1,2,figsize=(10,2.8),sharex=True,sharey=True)
dates=df_all['Date']

df_water = SpikeRemover(cutvalue=5,cutmode='zthresh').set_output(transform='pandas').fit_transform(df_water)

for i,(columnName,columnData) in enumerate(df_water.items()):
    zmean=np.mean(columnData.loc[lambda x : x != 0])
    # x,y=fig_indexes(2,i)
    axs[i].hlines(y=zmean,xmin=min(dates),xmax=max(dates), linewidth=0.5, color='r',linestyles='dashed',label='Mean')
    axs[i].plot(dates,columnData,label=columnName)
    axs[i].grid(visible=True,which='Major',axis='both')
    # if y > 0:
    axs[1].axvspan(datetime(2022,10,10), datetime(2022,12,31),color='g',alpha=0.2)
        # axs[x][y].axvspan(datetime(2023,10,20), datetime(2023,12,10),color='g',alpha=0.2)
        # axs[x][y].axvspan(datetime(2023,12,31), datetime(2024,1,31),color='g',alpha=0.2)
    # else:
    axs[0].axvspan(datetime(2022,10,10), datetime(2022,12,31),color='r',alpha=0.2)
        # axs[x][y].axvspan(datetime(2023,10,20), datetime(2023,12,10),color='r',alpha=0.2)
        # axs[x][y].axvspan(datetime(2023,12,31), datetime(2024,1,31),color='r',alpha=0.2)
    # if (x,y) == (0,0):
    axs[i].set_ylabel('Water Consumption', fontsize=10)
    # elif (x,y) == (1,0):
    #     axs[x][y].set_ylabel('Cold Consumption', fontsize=10)
    # if x > 0:
    axs[i].tick_params(axis='x', labelrotation=45)
    # elif y == 0:
    axs[i].set_title('Original',fontsize=12)
    # else:
    axs[i].set_title('Updated',fontsize=12)
    axs[i].set_yticklabels([])
plt.suptitle('Water Consumption Data Quality',fontsize=14)
plt.tight_layout()

fig.savefig('./Output Files/Images/Pre Processing/water_corrections.png',format='png',bbox_inches='tight')





### VPN And Webex Shift Correction

In [ ]:
fig,axs=plt.subplots(2,2,figsize=(10,4),sharex=True,sharey=True)
dates=df_all['Date']

df_vpn_webex = SpikeRemover(cutvalue=5,cutmode='zthresh').set_output(transform='pandas').fit_transform(df_vpn_webex)

for i,(columnName,columnData) in enumerate(df_vpn_webex.items()):
    zmean=np.mean(columnData.loc[lambda x : x != 0])
    x,y=fig_indexes(2,i)
    axs[x][y].hlines(y=zmean,xmin=min(dates),xmax=max(dates), linewidth=0.5, color='r',linestyles='dashed',label='Mean')
    axs[x][y].plot(dates,columnData,label=columnName)
    axs[x][y].grid(visible=True,which='Major',axis='both')
    if y > 0:
        axs[x][y].axvspan(datetime(2022,12,10), datetime(2023,4,30),color='g',alpha=0.2)
        # axs[x][y].axvspan(datetime(2023,10,20), datetime(2023,12,10),color='g',alpha=0.2)
        # axs[x][y].axvspan(datetime(2023,12,31), datetime(2024,1,31),color='g',alpha=0.2)
    else:
        axs[x][y].axvspan(datetime(2022,12,10), datetime(2023,4,30),color='r',alpha=0.2)
        # axs[x][y].axvspan(datetime(2023,10,20), datetime(2023,12,10),color='r',alpha=0.2)
        # axs[x][y].axvspan(datetime(2023,12,31), datetime(2024,1,31),color='r',alpha=0.2)
    if (x,y) == (0,0):
        axs[x][y].set_ylabel('VPN Connections', fontsize=10)
    elif (x,y) == (1,0):
        axs[x][y].set_ylabel('Meeting Connections', fontsize=10)
    if x > 0:
        axs[x][y].tick_params(axis='x', labelrotation=45)
    elif y == 0:
        axs[x][y].set_title('Original',fontsize=12)
    else:
        axs[x][y].set_title('Updated',fontsize=12)
    axs[x][y].set_yticklabels([])
plt.suptitle('VPN and Meeting Data Quality',fontsize=14)
plt.tight_layout()

fig.savefig('./Output Files/Images/Pre Processing/vpn_webex_corrections.png',format='png',bbox_inches='tight')





## Build Full Feature Set

In [ ]:
# # Build data sets
df_leading_only=df_all[[time_field]+leading_features+[label_field]]

df_lag1=df_all[[time_field]+lagging_features].add_suffix('_onedayago',axis=1)
df_lag1[time_field]=df_lag1[time_field+'_onedayago'].shift(-1)
lag1_features=df_lag1.columns[1:-1].tolist()
print(f'{len(lag1_features)} lag1 Features: {lag1_features}')

df_lag7=df_all[[time_field]+lagging_features].add_suffix('_oneweekago',axis=1)
df_lag7[time_field]=df_lag7[time_field+'_oneweekago'].shift(-7)
lag7_features=df_lag7.columns[1:-1].tolist()
print(f'{len(lag7_features)} lag7 Features: {lag7_features}')

df_plusLag1=df_leading_only.merge(right=df_lag1,how='left',on=time_field).drop(time_field+'_onedayago',axis=1)
df_plusLag7=df_plusLag1.merge(right=df_lag7,how='left',on=time_field).drop(time_field+'_oneweekago',axis=1)

df_plusLag7['EOMonth']=df_plusLag7['Date'] + pd.tseries.offsets.MonthEnd(0)
df_plusLag30=df_plusLag7.merge(right=df_waste,how='left',on='EOMonth').drop('EOMonth',axis=1)
df_plusLag7.drop('EOMonth',axis=1,inplace=True)

df_plusPeriodic=df_plusLag30.merge(right=df_all[[time_field]+periodic_features],how='left',on=time_field)
df_plusLaggedLabels=df_plusPeriodic.merge(right=df_ts,how='left',on=time_field)
df_plusLaggedLabels.columns = df_plusLaggedLabels.columns.astype(str)

df_plusLaggedLabels.info()

### Feature Set QC

In [ ]:
pds = MultiPipe()

In [ ]:
baseline_preproc=Pipeline(
    steps=[
        ('simple_impute',SimpleImputer()),
        ('std scale',StandardScaler())
    ]
)

pds.AddPreProc(baseline_preproc,'Simple')
pds.PurgeQCSet('Feature Eng')
pds.AddQCSet('Simple','Feature Eng')


CV = TimeSeriesSplit(gap=0, max_train_size=None, n_splits=10, test_size=10) # reduced length CV

# _ = pds.CalculateScores('Feature Eng','Simple','excl Desks Booked',df_leading_only.set_index(time_field).loc[:date_val_end].drop(columns=[label_field,'Desks_Booked']),df_leading_only.set_index(time_field).loc[:date_val_end][label_field])
_ = pds.CalculateScores('Feature Eng','Simple','Leading Only',df_leading_only.set_index(time_field).loc[:date_val_end].drop(columns=[label_field]),df_leading_only.set_index(time_field).loc[:date_val_end][label_field])
_ = pds.CalculateScores('Feature Eng','Simple','Plus Lag1 Feat',df_plusLag1.set_index(time_field).loc[:date_val_end].drop(columns=[label_field]+['Heat_Consumption_onedayago','Cold_Consumption_onedayago']),df_plusLag1.set_index(time_field).loc[:date_val_end][label_field])
_ = pds.CalculateScores('Feature Eng','Simple','Plus Lag7 Feat',df_plusLag7.set_index(time_field).loc[:date_val_end].drop(columns=[label_field]+['Heat_Consumption_onedayago','Cold_Consumption_onedayago','Heat_Consumption_oneweekago','Cold_Consumption_oneweekago']),df_plusLag7.set_index(time_field).loc[:date_val_end][label_field])
_ = pds.CalculateScores('Feature Eng','Simple','Plus Lag30 Feat',df_plusLag30.set_index(time_field).loc[:date_val_end].drop(columns=[label_field]+['Heat_Consumption_onedayago','Cold_Consumption_onedayago','Heat_Consumption_oneweekago','Cold_Consumption_oneweekago']),df_plusLag30.set_index(time_field).loc[:date_val_end][label_field])
_ = pds.CalculateScores('Feature Eng','Simple','Plus Periodic',df_plusPeriodic.set_index(time_field).loc[:date_val_end].drop(columns=[label_field]+['Heat_Consumption_onedayago','Cold_Consumption_onedayago','Heat_Consumption_oneweekago','Cold_Consumption_oneweekago']),df_plusPeriodic.set_index(time_field).loc[:date_val_end][label_field])
_ = pds.CalculateScores('Feature Eng','Simple','Plus AR',df_plusLaggedLabels.set_index(time_field).loc[:date_val_end].drop(columns=[label_field]+['Heat_Consumption_onedayago','Cold_Consumption_onedayago','Heat_Consumption_oneweekago','Cold_Consumption_oneweekago']),df_plusLaggedLabels.set_index(time_field).loc[:date_val_end][label_field])


print(len(df_plusLaggedLabels.set_index(time_field).loc[:date_val_end]))
print(len(df_plusLaggedLabels.set_index(time_field).loc[:date_val_end].dropna()))


In [ ]:
_ = pds.GetScores(metric_keys=['R^2 Score','RMS Error','Mean Absolute Error'],verbose=False)
fig,axs=plt.subplots(1,len(pds.active_metrics),figsize=(0.5+5*len(pds.active_metrics),4))  
pds.GraphScores('Feature Eng',axs)

# fig.savefig('./Output Files/Images/Feature Engineering/Feature_Set_Metrics.png',format='png',bbox_inches='tight')
axs[0].set_ylim(0.015,0.04)
axs[1].set_ylim(0.02,0.05)
axs[2].set_ylim(0.75,1.0)
fig.suptitle('Impact of Building Feature Set on Regression Model Accuracy',fontsize=12,fontweight='bold')
for ax in axs:
    ax.set_xlabel('Feature Set Added')
fig.tight_layout()
fig.savefig('./Output Files/Images/Pre Processing/Feature_Set_Metrics.png',format='png',bbox_inches='tight')

## Outlier Removal
### Despike and dezero preprocessors

In [ ]:
zthresh=5

outlier_feature_list=['Heat_Consumption','Cold_Consumption','Meeting_Cnxns','Meeting_Participants','FTE_Count','Meeting_Duration']
dezero_feature_list = ['VPN_cnxn','Meeting_Cnxns','Meeting_Participants','Concurrent_Meetings','Day_Electric_KWh','Night_Electric_KWh','Meeting_Duration']

outlier_feature_list_ext = [x + '_onedayago' for x in outlier_feature_list] + [x + '_oneweekago' for x in outlier_feature_list] 
dezero_feature_list_ext = [x + '_onedayago' for x in dezero_feature_list] + [x + '_oneweekago' for x in dezero_feature_list] 

despike_transformer=ColumnTransformer(
    transformers=[
        ('5z_despike',SpikeRemover(cutvalue=5,cutmode='zthresh'),outlier_feature_list_ext)
    ],
    remainder='passthrough',
    verbose_feature_names_out=False
)
dezero_transformer=ColumnTransformer(
    transformers=[
        ('dezero',SpikeRemover(cutvalue=0,cutmode='value'),dezero_feature_list_ext)
    ],
    remainder='passthrough',
    verbose_feature_names_out=False
)
# despike_preproc=Pipeline(
#     steps=[
#         ('despike',despike_transformer.set_output(transform='pandas')),
#         ('dezero',dezero_transformer.set_output(transform='pandas')),
#     ]
# )

df_all_dspike = despike_transformer.set_output(transform='pandas').fit_transform(df_plusLaggedLabels.set_index(time_field))
df_all_dzero = dezero_transformer.set_output(transform='pandas').fit_transform(df_all_dspike)

# not_enough_data_dz=[]
# for columnName, columnData in df_all_dzero.items():
#     if columnData.count() / len(columnData) < 0.8:
#         not_enough_data_dz.append(columnName)
# print(not_enough_data_dz)

# df_clip_col=df_all_dzero.drop(columns=not_enough_data_dz)

# print(df_clip_col[df_clip_col['Work_Days_Per_Month'] < 15].index)
df_clip_row1=df_all_dzero.drop(df_all_dzero[df_all_dzero['Work_Days_Per_Month'] < 15].index)
df_clip_row2=df_clip_row1.dropna(thresh=df_clip_row1.shape[1]-10)

# ax1 = msno.bar(df_all_dzero)

# ax1.plot()
# ax1.figure.savefig('./Output Files/Images/Pre Processing/missing_values_matrix_dspik.png',format='png',bbox_inches='tight')


### Despike QC

In [ ]:
pds.AddQCSet('Simple','Despike')

_ = pds.CalculateScores('Despike','Simple','No Despike',df_plusLaggedLabels.set_index(time_field).loc[:date_val_end].drop(columns=[label_field]+['Heat_Consumption_onedayago','Cold_Consumption_onedayago','Heat_Consumption_oneweekago','Cold_Consumption_oneweekago']),df_plusLaggedLabels.set_index(time_field).loc[:date_val_end][label_field])
_ = pds.CalculateScores('Despike','Simple','Despiked',df_all_dspike.loc[:date_val_end].drop(columns=label_field),df_all_dspike.loc[:date_val_end][label_field])
_ = pds.CalculateScores('Despike','Simple','Dezeroed',df_all_dzero.loc[:date_val_end].drop(columns=label_field),df_all_dzero.loc[:date_val_end][label_field])

# Xy_val=df_clip_col.loc[:date_val_end]
# _ = pds.CalculateScores('Impute Tests','DailyMean','Daily Mean\nclip col',Xy_val.drop(columns=[label_field]),Xy_val[label_field])
# Xy_val=df_clip_row1.loc[:date_val_end]
_ = pds.CalculateScores('Despike','Simple','Clip Partial Months',df_clip_row1.loc[:date_val_end].drop(columns=[label_field]),df_clip_row1.loc[:date_val_end][label_field])
# Xy_val=df_clip_row2.loc[:date_val_end]
_ = pds.CalculateScores('Despike','Simple','Clip Partial Rows',df_clip_row2.loc[:date_val_end].drop(columns=[label_field]),df_clip_row2.loc[:date_val_end][label_field])

# print(len(df_plusLaggedLabels.set_index(time_field).loc[:date_val_end]))
# print(len(df_all_dzero.loc[:date_val_end].dropna()))


In [ ]:
# _ = pds.GetScores(qc_set_keys=['Despike'] ,metric_keys=['R^Score','RMS Error','Mean Absolute Error'],verbose=False)
_ = pds.GetScores(qc_set_keys=['Despike'] ,metric_keys=['R^2 Score','RMS Error','Mean Absolute Error'],verbose=False, ValFilter=1)
fig,axs=plt.subplots(1,len(pds.active_metrics),figsize=(0.5+5*len(pds.active_metrics),4))  
pds.GraphScores(qc_set_key='Despike',axs=axs)

axs[0].set_ylim(0.015,0.035)
axs[1].set_ylim(0.02,0.045)
axs[2].set_ylim(0.8,1.0)
fig.suptitle('Impact of Bad Value Removal Set on Regression Model Accuracy',fontsize=12,fontweight='bold')
for ax in axs:
    ax.set_xlabel('Values Removed')
fig.tight_layout()

# for figax in [axs]:
#     # axs[0].set_ylim(0.75,1)
#     figax[0].set_ylim(0.0,0.2)
#     figax[1].set_ylim(0.0,0.2)
#     figax[2].set_ylim(0.0,1.0)

fig.savefig('./Output Files/Images/Pre Processing/Dspike_Metrics.png',format='png',bbox_inches='tight')

## Imputation

In [ ]:
# fig1 = msno.matrix(df_all_dzero)
# fig1.plt
# fig1.figure.savefig('./Output Files/Images/Pre Processing/missing_val_matrix_before_imputation.png',format='png',bbox_inches='tight')

# ax1 = msno.matrix(df_all_dzero,figsize=(20,5))
# ax1.plot()
# ax1.figure.savefig('./Output Files/Images/Data Exploration/missing_val_matrix_before_imputation.png',format='png',bbox_inches='tight')

# ax2 = msno.matrix(DailyMeanImputer(groupby_name='Day_Of_Week').fit_transform(df_all_dzero),figsize=(20,5))
# ax2.plot()
# ax2.figure.savefig('./Output Files/Images/Data Exploration/missing_val_matrix_after_imputation.png',format='png',bbox_inches='tight')

### Impute Preprocessors

In [ ]:
NoImpute = Pipeline(
    steps=[
        # ('simple mean',SimpleImputer()),
        ('std scale',StandardScaler())
    ]
)
pds.AddPreProc(NoImpute,'NoImpute')
SimpleImpute = Pipeline(
    steps=[
        ('simple mean',SimpleImputer()),
        ('std scale',StandardScaler())
    ]
)
pds.AddPreProc(SimpleImpute,'SimpleMean')
DailyImpute = Pipeline(
    steps=[
        ('daily mean',DailyMeanImputer(groupby_name='Day_Of_Week')),
        ('std scale',StandardScaler())
    ]
)
pds.AddPreProc(DailyImpute,'DailyMean')
DailyImpute = Pipeline(
    steps=[
        ('daily rolling',DailyMeanImputer(groupby_name='Day_Of_Week',rolling=7)),
        ('daily mean',DailyMeanImputer(groupby_name='Day_Of_Week')),
        ('std scale',StandardScaler())
    ]
)
pds.AddPreProc(DailyImpute,'DailyRolling')



# msno.matrix(df_clip_row1)
# DailyImpute.fit(df_clip_row1.loc[:date_val_end])
# msno.matrix(DailyImpute.set_output(transform='pandas').transform(df_clip_row1))

# msno.matrix(DailyImpute.fit_transform(df_clip_row1))


### Impute QC

In [ ]:

# pds.CV=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=4, test_size=40)
pds.PurgeQCSet('Impute Tests')

pds.AddQCSet('NoImpute','Impute Tests')
pds.AddQCSet('Simple','Impute Tests')
pds.AddQCSet('SimpleMean','Impute Tests')
pds.AddQCSet('DailyMean','Impute Tests')
pds.AddQCSet('DailyRolling','Impute Tests')

Xy_val=df_clip_row1.loc[:date_val_end].copy()
# Xy_val.drop(columns='Month',inplace=True)
# pds.CV=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=4, test_size=40) 
_ = pds.CalculateScores('Impute Tests','NoImpute','Drop NA',Xy_val.dropna().drop(columns=label_field),Xy_val.dropna()[label_field])
_ = pds.CalculateScores('Impute Tests','Simple','Fill Forward',Xy_val.ffill().bfill().drop(columns=label_field),Xy_val.ffill().bfill()[label_field])
_ = pds.CalculateScores('Impute Tests','SimpleMean','Column Mean',Xy_val.drop(columns=label_field),Xy_val[label_field])
_ = pds.CalculateScores('Impute Tests','DailyMean','Daily Mean',Xy_val.drop(columns=label_field),Xy_val[label_field])
_ = pds.CalculateScores('Impute Tests','DailyRolling','Rolling Mean',Xy_val.drop(columns=label_field),Xy_val[label_field])


In [ ]:
temp = pds.GetScores(qc_set_keys=['Impute Tests'],metric_keys=['R^2 Score','RMS Error','Mean Absolute Error'],verbose=False,ValFilter=1)
fig,axs=plt.subplots(1,len(pds.active_metrics),figsize=(0.5+5*len(pds.active_metrics),4))  
pds.GraphScores(qc_set_key='Impute Tests',axs=axs)
axs[0].set_ylim(0.015,0.035)
axs[1].set_ylim(0.02,0.045)
axs[2].set_ylim(0.8,1.0)
fig.suptitle('Impact of Imputation Method on Regression Model Accuracy',fontsize=12,fontweight='bold')
for ax in axs:
    ax.set_xlabel('Values Removed')
fig.tight_layout()
fig.savefig('./Output Files/Images/Pre Processing/Imputation_Metrics.png',format='png',bbox_inches='tight')

### Create imputed dataset

In [ ]:
dmi=DailyMeanImputer(groupby_name='Day_Of_Week')
dmi.fit(df_clip_row1.loc[:date_val_end])
df_imputed = dmi.set_output(transform='pandas').transform(df_clip_row1)

lagging_features_ext = [x + '_onedayago' for x in lagging_features]+[x + '_oneweekago' for x in lagging_features]
lagging_features_ext = [x for x in lagging_features_ext if x not in not_enough_data_dz]

with engine.connect() as conn:
    # Re-ordering fields on export to make recreating feature groups easier on future import
    df_imputed[leading_features+lagging_features_ext+waste_features+periodic_features+timeseries_features+[label_field]].to_sql('Preprocessed_Features',conn,schema='Gold',if_exists='replace',dtype=sqlcol(df_imputed))

### Despike and Imputation Time Series Analysis

In [ ]:
badspikes = [x + '_onedayago' for x in outlier_feature_list] 
badzeroes = [x + '_onedayago' for x in dezero_feature_list]
badlist = badspikes + badzeroes
# badlist = [x for x in badlist if x not in not_enough_data_dz]
fig,axs=plt.subplots(len(badlist),3,figsize=(14,3*len(badlist)),sharex=True)  
dates = df_imputed.index
for i,colname in enumerate(badlist):
    axs[i][0].plot(dates,df_plusLaggedLabels.set_index('Date').loc[df_imputed.index][colname])
    #axs[i][1].plot(dates,df_all_dspike[colname])
    axs[i][1].plot(dates,df_all_dzero.loc[df_imputed.index][colname])
    axs[i][2].plot(dates,df_imputed[colname])

    zmean=np.mean(df_plusLaggedLabels[colname].loc[lambda x : x != 0])
    zstd=np.std(df_plusLaggedLabels[colname].loc[lambda x : x != 0])
    zdist = np.abs(df_plusLaggedLabels[colname] - zmean) / zstd

    maskz = (zthresh <= zdist) if colname in badspikes else [False for x in zdist]
    mask0 = (df_plusLaggedLabels[colname] == 0) if colname in badzeroes else [False for x in df_plusLaggedLabels[colname]]
    zoutliers = np.sum(maskz)
    zeroes = np.sum(mask0)

    axs[i][0].set_title(f'{colname}\nOriginal',fontsize=10)
    axs[i][1].set_title(f'{colname}\n{zoutliers+zeroes} value errors removed',fontsize=10)
    axs[i][2].set_title(f'{colname}\nAfter Daily Mean Imputation',fontsize=10)


    for j in range(0,3):
        axs[i][j].hlines(y=zmean+zthresh*zstd,xmin=min(dates),xmax=max(dates), linewidth=1, color='r',linestyles='dashed')
        axs[i][j].hlines(y=zmean-zthresh*zstd,xmin=min(dates),xmax=max(dates), linewidth=1, color='r',linestyles='dashed')
        axs[i][j].hlines(y=0,xmin=min(dates),xmax=max(dates), linewidth=1, color='m',linestyles='dashed')
        axs[i][j].set_ylim(1.1*(zmean-zthresh*zstd),1.1*(zmean+zthresh*zstd))
        axs[i][j].set_xlim(min(dates),max(dates))
        axs[i][j].grid(visible=True,which='Major',axis='both')
        axs[i][j].set_yticklabels([])
        axs[i][j].axvspan(date_test_start, df_imputed.index[-1],color='c',alpha=0.2)



axs[i][0].tick_params(axis='x', labelrotation=45)
axs[i][1].tick_params(axis='x', labelrotation=45)
axs[i][2].tick_params(axis='x', labelrotation=45)


# fig.savefig('./Output Files/Images/Pre Processing/Despike_Impute_timeseries.png',format='png',bbox_inches='tight')


In [ ]:
badspikes = [x + '_onedayago' for x in outlier_feature_list] 
badzeroes = [x + '_onedayago' for x in dezero_feature_list]
badlist = badspikes + badzeroes
# badlist = [x for x in badlist if x not in not_enough_data_dz]
fig,axs=plt.subplots(3,1,figsize=(6,6),sharey=True,sharex=True)  
dates = df_imputed.index
for i,colname in enumerate(['Meeting_Participants_onedayago']):
    axs[0].plot(dates,df_plusLaggedLabels.set_index('Date').loc[df_imputed.index][colname])
    #axs[1].plot(dates,df_all_dspike[colname])
    axs[1].plot(dates,df_all_dzero.loc[df_imputed.index][colname])
    axs[2].plot(dates,df_imputed[colname])

    zmean=np.mean(df_plusLaggedLabels[colname].loc[lambda x : x != 0])
    zstd=np.std(df_plusLaggedLabels[colname].loc[lambda x : x != 0])
    zdist = np.abs(df_plusLaggedLabels[colname] - zmean) / zstd

    maskz = (zthresh <= zdist) if colname in badspikes else [False for x in zdist]
    mask0 = (df_plusLaggedLabels[colname] == 0) if colname in badzeroes else [False for x in df_plusLaggedLabels[colname]]
    zoutliers = np.sum(maskz)
    zeroes = np.sum(mask0)

    axs[0].set_title(f'Original',fontsize=10)
    axs[1].set_title(f'{zoutliers+zeroes} Value Errors Removed',fontsize=10)
    axs[2].set_title(f'After Daily Mean Imputation',fontsize=10)


    for j in range(0,3):
        # axs[j].hlines(y=zmean+zthresh*zstd,xmin=min(dates),xmax=max(dates), linewidth=1, color='r',linestyles='dashed')
        # axs[j].hlines(y=zmean-zthresh*zstd,xmin=min(dates),xmax=max(dates), linewidth=1, color='r',linestyles='dashed')
        # axs[j].hlines(y=0,xmin=min(dates),xmax=max(dates), linewidth=1, color='m',linestyles='dashed')
        axs[j].set_ylim(1.1*(zmean-2*zstd),1.1*(zmean+2*zstd))
        axs[j].set_xlim(min(dates),max(dates))
        axs[j].grid(visible=True,which='Major',axis='both')
        axs[j].set_yticklabels([])
        axs[j].axvspan(date_test_start, df_imputed.index[-1],color='c',alpha=0.1)


        axs[j].set_ylabel('Participants')

# axs[0].tick_params(axis='x', labelrotation=45)
# axs[1].tick_params(axis='x', labelrotation=45)
axs[2].tick_params(axis='x', labelrotation=45)

fig.suptitle('Bad Value Removal and Imputation: Meeting Participants')
fig.tight_layout()
fig.savefig('./Output Files/Images/Pre Processing/Despike_Impute_timeseries.png',format='png',bbox_inches='tight')

## Cyclical encoding and Scaling Test

### Scaling Transformers

In [ ]:
std_scale=Pipeline(
    steps=[
        ('std scale',StandardScaler()),
    ]
)
mm_scale=Pipeline(
    steps=[
        ('mm scale',MinMaxScaler()),
    ]
)
rob_scale=Pipeline(
    steps=[
        ('rob scale',RobustScaler()),
    ]
)

pds.AddPreProc(std_scale,'Std')
pds.AddPreProc(mm_scale,'MinMax')
pds.AddPreProc(rob_scale,'Robust')

pds.AddQCSet('Std','Scale Tests')
pds.AddQCSet('MinMax','Scale Tests')
pds.AddQCSet('Robust','Scale Tests')

Xy_val=df_imputed.loc[:date_val_end].copy()
# pds.CV=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=4, test_size=40) 
_ = pds.CalculateScores('Scale Tests','Std','Standard',Xy_val.drop(columns=label_field),Xy_val[label_field])
_ = pds.CalculateScores('Scale Tests','MinMax','Min Max',Xy_val.dropna().drop(columns=label_field),Xy_val[label_field])
_ = pds.CalculateScores('Scale Tests','Robust','Robust',Xy_val.drop(columns=label_field),Xy_val[label_field])

### Scaling Metrics

In [ ]:
temp = pds.GetScores(qc_set_keys=['Scale Tests'],metric_keys=['R^2 Score','RMS Error','Mean Absolute Error'],verbose=False)
fig,axs=plt.subplots(1,len(pds.active_metrics),figsize=(0.5+5*len(pds.active_metrics),4))  
pds.GraphScores(qc_set_key='Scale Tests',axs=axs)
axs[0].set_ylim(0.015,0.035)
axs[1].set_ylim(0.02,0.045)
axs[2].set_ylim(0.8,1.0)
fig.suptitle('Impact of Scaling Method on Regression Model Accuracy',fontsize=12,fontweight='bold')
fig.tight_layout()

fig.savefig('./Output Files/Images/Pre Processing/Scaler_Metrics.png',format='png',bbox_inches='tight')

### Cyclic Encoding Transformers

In [ ]:
# https://scikit-learn.org/stable/auto_examples/applications/plot_cyclical_feature_engineering.html
# from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, SplineTransformer

def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))


def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))


ohe_transformer=ColumnTransformer(
    transformers=[
        ('ohe',OneHotEncoder(sparse_output=False,drop='if_binary'),['Day_Of_Week','Quarter','School_Holiday']),
        # ('ohe_m',OneHotEncoder(sparse_output=False,categories=[x for x in range(1,13)]),['Month']),
        # ('ohe_d',OneHotEncoder(sparse_output=False,categories=[x for x in range(1,5)]),['Day_Of_Week']),
        # ('bin',OneHotEncoder(sparse_output=False),[])
    ],
    remainder=StandardScaler(),
    # remainder='drop',
    verbose_feature_names_out=False
)

cyclic_trig_transformer = ColumnTransformer(
    transformers=[
        ("day_sin", sin_transformer(5), ['Day_Of_Week']),
        ("day_cos", cos_transformer(5), ['Day_Of_Week']),
        # ("quarter_sin", sin_transformer(4), ['Quarter']),
        # ("quarter_cos", cos_transformer(4), ['Quarter']),
        ("month_sin", sin_transformer(12), ['Month']),
        ("month_cos", cos_transformer(12), ['Month']),
        ('bin',OneHotEncoder(sparse_output=False,drop='if_binary'),['School_Holiday'])
    ],
    verbose_feature_names_out=False,
    remainder=StandardScaler(),
)

cyclic_trig_transformer_Q = ColumnTransformer(
    transformers=[
        ("day_sin", sin_transformer(5), ['Day_Of_Week']),
        ("day_cos", cos_transformer(5), ['Day_Of_Week']),
        ("quarter_sin", sin_transformer(4), ['Quarter']),
        ("quarter_cos", cos_transformer(4), ['Quarter']),
        # ("month_sin", sin_transformer(12), ['Month']),
        # ("month_cos", cos_transformer(12), ['Month']),
        ('bin',OneHotEncoder(sparse_output=False,drop='if_binary'),['School_Holiday'])
    ],
    verbose_feature_names_out=False,
    remainder=StandardScaler(),
)

def periodic_spline_transformer(period, n_splines=None, degree=3):
    if n_splines is None:
        n_splines = period
    n_knots = n_splines + 1  # periodic and include_bias is True
    return SplineTransformer(
        degree=degree,
        n_knots=n_knots,
        knots=np.linspace(0, period, n_knots).reshape(n_knots, 1),
        extrapolation="periodic",
        include_bias=True,
    )


cyclic_spline_transformer = ColumnTransformer(
    transformers=[
        ("cyclic_weekday", periodic_spline_transformer(5, n_splines=3), ['Day_Of_Week']),
        # ("cyclic_quarter", periodic_spline_transformer(4, n_splines=3), ['Quarter']),
        ("cyclic_month", periodic_spline_transformer(12, n_splines=6), ['Month']),
        ('bin',OneHotEncoder(sparse_output=False,drop='if_binary'),['School_Holiday'])
    ],
    remainder=StandardScaler(),
)
cyclic_spline_transformer_Q = ColumnTransformer(
    transformers=[
        ("cyclic_weekday", periodic_spline_transformer(5, n_splines=3), ['Day_Of_Week']),
        ("cyclic_quarter", periodic_spline_transformer(4, n_splines=3), ['Quarter']),
        # ("cyclic_month", periodic_spline_transformer(12, n_splines=6), ['Month']),
        ('bin',OneHotEncoder(sparse_output=False,drop='if_binary'),['School_Holiday'])
    ],
    remainder=StandardScaler(),
)



pds.AddPreProc(ohe_transformer,'ohe')
# pds.AddPreProc(cyclic_trig_transformer,'cossin')
pds.AddPreProc(cyclic_trig_transformer_Q,'cossin')
# pds.AddPreProc(cyclic_spline_transformer,'spline')
pds.AddPreProc(cyclic_spline_transformer_Q,'spline')
# pds.AddPreProc(pipe=None,key='empty')

# pds.AddPreProc(make_pipeline(DailyMeanImputer(groupby_name='Day_Of_Week'),std_scale),'Std')
# pds.AddPreProc(make_pipeline(DailyMeanImputer(groupby_name='Day_Of_Week'),ohe_transformer),'ohe')
# # pds.AddPreProc(make_pipeline(DailyMeanImputer(groupby_name='Day_Of_Week'),cyclic_trig_transformer),'cossin')
# pds.AddPreProc(make_pipeline(DailyMeanImputer(groupby_name='Day_Of_Week'),cyclic_trig_transformer_Q),'cossin')
# # pds.AddPreProc(make_pipeline(DailyMeanImputer(groupby_name='Day_Of_Week'),cyclic_spline_transformer),'spline')
# pds.AddPreProc(make_pipeline(DailyMeanImputer(groupby_name='Day_Of_Week'),cyclic_spline_transformer_Q),'spline')
# pds.AddPreProc(make_pipeline(DailyMeanImputer(groupby_name='Day_Of_Week')),key='empty')


pds.PurgeQCSet('Cyclic Tests')
pds.AddQCSet('Std','Cyclic Tests')
pds.AddQCSet('ohe','Cyclic Tests')
pds.AddQCSet('cossin','Cyclic Tests')
# pds.AddQCSet('cossin_q','Cyclic Tests')
pds.AddQCSet('spline','Cyclic Tests')
# pds.AddQCSet('spline_q','Cyclic Tests')
# pds.AddQCSet('empty','Cyclic Tests')


### Cyclic encoding metrics

In [ ]:
Xy_val = df_imputed.loc[:date_val_end].copy()#.set_index('Date')
# colorder = Xy_val.columns.tolist()

# if 'Quarter' in colorder:
#     old='Quarter'
#     new='Month'

#     for i,val in enumerate(colorder):
#         if val=='Quarter':
#             break
#     neworder=colorder[:i] + ['Month'] + colorder[i+1:]
    
#     _ = pds.CalculateScores('Cyclic Tests','spline_q','Splines_Q',Xy_val.drop(columns=label_field),Xy_val[label_field])
#     _ = pds.CalculateScores('Cyclic Tests','cossin_q','Cos-Sin_Q',Xy_val.drop(columns=label_field),Xy_val[label_field])

#     Xy_val = df_clip_row1.copy().loc[:date_val_end].drop(columns='Quarter')
#     Xy_val['Month'] = Xy_val.index.month
#     Xy_val = Xy_val[neworder]
#     print(Xy_val.columns.tolist())
#     _ = pds.CalculateScores('Cyclic Tests','Std','Scale Only',Xy_val.drop(columns=label_field),Xy_val[label_field])
#     _ = pds.CalculateScores('Cyclic Tests','empty','OHE',Xy_val.drop(columns=label_field),Xy_val[label_field])
#     _ = pds.CalculateScores('Cyclic Tests','cossin','Cos-Sin',Xy_val.drop(columns=label_field),Xy_val[label_field])
#     _ = pds.CalculateScores('Cyclic Tests','spline','Splines',Xy_val.drop(columns=label_field),Xy_val[label_field])

# else:

#     for i,val in enumerate(colorder):
#         if val=='Month':
#             break
#     neworder=colorder[:i] + ['Quarter'] + colorder[i+1:]


_ = pds.CalculateScores('Cyclic Tests','Std','Scale Only',Xy_val.drop(columns=label_field),Xy_val[label_field])
# X_ohe=ohe_transformer.fit_transform(Xy_val.drop(columns=label_field))
_ = pds.CalculateScores('Cyclic Tests','ohe','OHE',Xy_val.drop(columns=label_field),Xy_val[label_field])
_ = pds.CalculateScores('Cyclic Tests','cossin','Cos-Sin',Xy_val.drop(columns=label_field),Xy_val[label_field])
_ = pds.CalculateScores('Cyclic Tests','spline','Splines',Xy_val.drop(columns=label_field),Xy_val[label_field])

    # Xy_val = df_clip_row1.copy().loc[:date_val_end].drop(columns='Month')
    # Xy_val['Quarter'] = (Xy_val.index.month // 4) + 1
    # Xy_val = Xy_val[neworder]
    # print(Xy_val.columns.tolist())

    # _ = pds.CalculateScores('Cyclic Tests','spline_q','Splines_Q',Xy_val.drop(columns=label_field),Xy_val[label_field])
    # _ = pds.CalculateScores('Cyclic Tests','cossin_q','Cos-Sin_Q',Xy_val.drop(columns=label_field),Xy_val[label_field])
    


In [ ]:
temp = pds.GetScores(qc_set_keys=['Cyclic Tests'],metric_keys=['R^2 Score','RMS Error','Mean Absolute Error'],verbose=False)
fig,axs=plt.subplots(1,len(pds.active_metrics),figsize=(0.5+5*len(pds.active_metrics),4))  
pds.GraphScores(qc_set_key='Cyclic Tests',axs=axs)
axs[0].set_ylim(0.015,0.035)
axs[1].set_ylim(0.02,0.045)
axs[2].set_ylim(0.8,1.0)
fig.suptitle('Impact of Value Encoding on Regression Model Accuracy',fontsize=12,fontweight='bold')
fig.tight_layout()

fig.savefig('./Output Files/Images/Feature Engineering/Ordinal_Encoding_Metrics.png',format='png',bbox_inches='tight')

## Added Noise Tests for Desks Booked

In [ ]:
# Credit: https://stackoverflow.com/questions/18441779/how-to-specify-upper-and-lower-limits-when-using-numpy-random-normal

lower = 0.0
upper = 2.0
mu = 0.95
sigma = 0.2

rangen=scipy.stats.truncnorm((lower-mu)/sigma,(upper-mu)/sigma,loc=mu,scale=sigma)

fig,ax=plt.subplots(1,1,figsize=(6,4))

samples = rangen.rvs(10000)
count, bins, ignored = plt.hist(samples, 30, density=True)
ax.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) *
               np.exp( - (bins - mu)**2 / (2 * sigma**2) ),
         linewidth=2, color='r')
#plt.show()
ax.set_title('Example Distribution - Mean 0.95, Sigma 0.1')
fig.savefig('./Output Files/Images/Feature Engineering/Sample Distribution.png',format='png',bbox_inches='tight')

In [ ]:
# X_ori=df_imputed.sort_index().loc[:date_val_end].drop(columns=label_field)
# print(X_ori['Desks_Booked'])
# print(df_pre_noise['Desks_Booked'])
# df_pre_noise['temp']=X_ori['Desks_Booked']
# df_pre_noise.head(40)

In [ ]:

# with engine.connect() as conn:
#     df_preproc = pd.read_sql_table('All_Raw_Features', conn,schema='Silver')

# df_preproc.set_index('Date',inplace=True)

X_ori=df_imputed.copy().sort_index().loc[:date_val_end].drop(columns=label_field)
# print(X_ori['Desks_Booked'])
X_ori['Desks_Booked']=df_pre_noise['Desks_Booked']
# print(X_ori['Desks_Booked'])
#X_ori.drop(columns='Month',inplace=True)
y=df_imputed[label_field].sort_index().loc[:date_val_end]

tra=filtered_transformer(X_ori.columns.tolist())
# pds = MultiPipe()
pds.AddPreProc(tra,'preproc')
pds.AddQCSet('preproc','Desks Booked Noise')
_ = pds.CalculateScores('Desks Booked Noise','preproc','Base',X_ori,y)

for sigma in [0.1,0.2]:
    for mu in [0.85,0.9,0.95,1.0]:
        rangen=scipy.stats.truncnorm((lower-mu)/sigma,(upper-mu)/sigma,loc=mu,scale=sigma)
        X=X_ori.copy()
        X['Desks_Booked']=X['Desks_Booked'].apply(lambda x: int(x*rangen.rvs(1)[0]))
        _ = pds.CalculateScores('Desks Booked Noise','preproc',f'{mu}' + u'\u00B1' + f'{sigma}',X,y)

In [ ]:
_ = pds.GetScores(qc_set_keys=['Desks Booked Noise'],metric_keys=['R^2 Score','RMS Error','Mean Absolute Error'],verbose=False)
fig,axs=plt.subplots(1,len(pds.active_metrics),figsize=(0.5+5*len(pds.active_metrics),4))  
pds.GraphScores(qc_set_key='Desks Booked Noise',axs=axs)
fig.suptitle('Impact of Adding Gaussian Noise to Desks Booked',fontsize=12,fontweight='bold')

axs[0].set_ylim(0.015,0.04)
axs[1].set_ylim(0.02,0.045)
axs[2].set_ylim(0.8,1.0)

for ax in axs:
    ax.tick_params(axis='x', labelrotation=90)
    ax.set_xlabel('Mean +/- Std Dev of Added Noise')
fig.tight_layout()
fig.savefig('./Output Files/Images/Feature Engineering/AddedNoise_metrics.png',format='png',bbox_inches='tight')



In [ ]:
to_apply_noise = ['Desks_Booked','Annual_Leave','Flexi_Leave','Total_Leave']
for col in to_apply_noise:
    print(f'Prenoise:  std.dev for {col}: {np.std(df_pre_noise[col]):.2f}, std.err: {np.std(df_pre_noise[col])/(len(to_apply_noise))**0.5:.2f}')
    print(f'Postnoise: std.dev for {col}: {np.std(df_imputed[col]):.2f}, std.err: {np.std(df_imputed[col])/(len(to_apply_noise))**0.5:.2f}')

df = df_imputed[['Pct_On_Site','Desks_Booked']].copy()
# display(df.head(40))
df[['Desks_Booked_PreNoise','Total_Staff','Desks_Booked_Scl']]=np.NaN
df.loc[:,'Desks_Booked_PreNoise']=df_pre_noise['Desks_Booked']
df.loc[:,'Total_Staff']=df_discarded.set_index('Date')['Total_Staff']
df.loc[:,'Desks_Booked_Scl']=(df['Desks_Booked']/df['Total_Staff'])/0.297*0.31
# df['Desks_Booked_PreNoise']=df_pre_noise['Desks_Booked']
# df['Total_Staff']=df_discarded['Total_Staff']

# display(df)
df_training= df.loc[:date_val_end]
# display(df_training.head(40))
metric_rmse_noise=root_mean_squared_error(df_training['Pct_On_Site'],df_training['Desks_Booked']/df_training['Total_Staff'])
metric_rmse_prenoise=root_mean_squared_error(df_training['Pct_On_Site'],df_training['Desks_Booked_PreNoise']/df_training['Total_Staff'])
print(f"Training Mean Actual {np.mean(df_training['Pct_On_Site']):.3f}, Training Mean Booked {np.mean(df_training['Desks_Booked']/df_training['Total_Staff']):.3f} Training Mean Booked w/Noise {np.mean(df_training['Desks_Booked_PreNoise']/df_training['Total_Staff']):.3f}")
print(metric_rmse_noise)
print(metric_rmse_prenoise)
print(root_mean_squared_error(df_training['Pct_On_Site'],df_training['Desks_Booked_Scl']))

df_test= df.loc[date_test_start:]
# display(df_test.head(40))
metric_rmse_noise=root_mean_squared_error(df_test['Pct_On_Site'],df_test['Desks_Booked']/df_test['Total_Staff'])
metric_rmse_prenoise=root_mean_squared_error(df_test['Pct_On_Site'],df_test['Desks_Booked_PreNoise']/df_test['Total_Staff'])
print(metric_rmse_noise)
print(metric_rmse_prenoise)
print(root_mean_squared_error(df_test['Pct_On_Site'],df_test['Desks_Booked_Scl']))
print(f"Test Mean Actual {np.mean(df_test['Pct_On_Site']):.3f}, Test Mean Booked {np.mean(df_test['Desks_Booked']/df_test['Total_Staff']):.3f} Test Mean Booked w/Noise {np.mean(df_test['Desks_Booked_PreNoise']/df_test['Total_Staff']):.3f}")


# for day in np.sort(df_imputed['Day_Of_Week'].unique()):
#     print(f'Day {day}')
#     print(f'Accuracy of Desks Booked, pre-noise: ')
#     print(f'Accuracy of Desks Booked, post-noise:')


In [ ]:
df_pre_noise

In [ ]:
from matplotlib import cm
from joypy import joyplot

df_all['Day_Of_Week']=df_all['Date'].dt.day_of_week



lower = 0.0
upper = 2.0
mu = 0.95
sigma = 0.2

rangen1=scipy.stats.truncnorm((lower-mu)/0.1,(upper-mu)/0.1,loc=mu,scale=0.1)
rangen2=scipy.stats.truncnorm((lower-mu)/0.2,(upper-mu)/0.2,loc=mu,scale=0.2)

X = pd.DataFrame(df_pre_noise['Desks_Booked'].copy())
X.rename(columns={'Desks_Booked':'Original'},inplace=True)

X['With SD 0.1']=X['Original'].apply(lambda x: int(x*rangen1.rvs(1)[0]))
X['With SD 0.2']=X['Original'].apply(lambda x: int(x*rangen2.rvs(1)[0]))

X.melt()

# display(df_all[['Day_Of_Week','Day']].sort_values('Day_Of_Week'))
# df_g=df_all.groupby('Day',sort=False)
fig, axes = joyplot(X.melt(), column='value',by='variable', colormap=cm.Pastel1, grid="y", overlap = 3, fade=False,title='Desks Booked Distribution with Noise',linewidth=1,figsize=(6,5))



for ax in axes:
    if Config.MASK_VALUE:
        ax.set_xticklabels([])
    ax.set_xlabel('Desks Booked')

fig.savefig('./Output Files/Images/Feature Engineering/AddedNoise_joy.png',format='png',bbox_inches='tight')

In [ ]:
to_apply_noise = ['Desks_Booked','Annual_Leave','Flexi_Leave','Total_Leave']
# to_apply_noise = ['Desks_Booked','Total_Leave']

lower = 0.0
upper = 2.0
mu = 0.95
sigma = 0.2

rangen1=scipy.stats.truncnorm((lower-mu)/0.1,(upper-mu)/0.1,loc=mu,scale=0.1)
rangen2=scipy.stats.truncnorm((lower-mu)/0.2,(upper-mu)/0.2,loc=mu,scale=0.2)

df_noised = df_imputed[to_apply_noise+['Day_Of_Week']].copy()
df_noised[to_apply_noise].astype("float64")
for field in to_apply_noise:
    df_noised[field]=df_noised[field].apply(lambda x: int(x*rangen1.rvs(1)[0]))


edge_days=[1,5]
core_days=[2,3,4]

# fig,axs=plt.subplots(2,1,figsize=(8,8))
# sns.histplot(data=df_imputed.loc[df_imputed['Day_Of_Week'].isin(edge_days)],x='Desks_Booked',ax=axs[0],kde=True,bins=25,stat='frequency',element='step',label='Original')
# sns.histplot(data=df_noised.loc[df_noised['Day_Of_Week'].isin(edge_days)],x='Desks_Booked',ax=axs[0],kde=True,bins=25,stat='frequency',element='step',label='With Noise s0.1')

# sns.histplot(data=df_imputed.loc[df_imputed['Day_Of_Week'].isin(core_days)],x='Desks_Booked',ax=axs[1],kde=True,bins=25,stat='frequency',element='step',label='Original')
# sns.histplot(data=df_noised.loc[df_noised['Day_Of_Week'].isin(core_days)],x='Desks_Booked',ax=axs[1],kde=True,bins=25,stat='frequency',element='step',label='With Noise s0.1')

# df_noised = df_imputed.copy()
# df_noised['Desks_Booked']=df_noised['Desks_Booked'].apply(lambda x: int(x*rangen2.rvs(1)[0]))

# sns.histplot(data=df_noised.loc[df_noised['Day_Of_Week'].isin(edge_days)],x='Desks_Booked',ax=axs[0],kde=True,bins=25,stat='frequency',element='step',label='With Noise s0.2')
# sns.histplot(data=df_noised.loc[df_noised['Day_Of_Week'].isin(core_days)],x='Desks_Booked',ax=axs[1],kde=True,bins=25,stat='frequency',element='step',label='With Noise s0.2')
# _ = ax.legend()

# display(df_noised.head(40))
# df_noised.info()

fig,axs=plt.subplots(len(to_apply_noise),2,figsize=(12,3.5*len(to_apply_noise)))

for i,field in enumerate(to_apply_noise):

    sns.kdeplot(data=df_imputed.loc[df_imputed['Day_Of_Week'].isin(edge_days)],x=field,ax=axs[i][0],label='Original',warn_singular=False)
    sns.kdeplot(data=df_noised.loc[df_noised['Day_Of_Week'].isin(edge_days)],x=field,ax=axs[i][0],label='With Noise s0.1')

    sns.kdeplot(data=df_imputed.loc[df_imputed['Day_Of_Week'].isin(core_days)],x=field,ax=axs[i][1],label='Original')
    sns.kdeplot(data=df_noised.loc[df_noised['Day_Of_Week'].isin(core_days)],x=field,ax=axs[i][1],label='With Noise s0.1')


df_noised = df_imputed.copy()
for field in to_apply_noise:
    df_noised[field]=df_noised[field].apply(lambda x: int(x*rangen2.rvs(1)[0]))

for i,field in enumerate(to_apply_noise):

    sns.kdeplot(data=df_noised.loc[df_noised['Day_Of_Week'].isin(edge_days)],x=field,ax=axs[i][0],label='With Noise s0.2')
    sns.kdeplot(data=df_noised.loc[df_noised['Day_Of_Week'].isin(core_days)],x=field,ax=axs[i][1],label='With Noise s0.2')

    _ = axs[i][0].set_title(f'{field} Distribution: Edge Days')
    _ = axs[i][1].set_title(f'{field} Distribution: Core Days')

    if Config.MASK_VALUE:
        axs[i][0].set_xticklabels([])
        axs[i][1].set_xticklabels([])
        axs[i][0].set_xlabel('')
        axs[i][1].set_xlabel('')


_ = axs[0][1].legend()

fig.tight_layout()
fig.savefig('./Output Files/Images/Feature Engineering/Shifted Distribution.png',format='png',bbox_inches='tight')



## Cross Validation Tests

In [ ]:
SVR_pipe = Pipeline(
    steps=[
        ('daily mean',SimpleImputer()),
        ('std scale',StandardScaler()),
        ('reg',LinearSVR())
    ]
)
LR_pipe = Pipeline(
    steps=[
        ('daily mean',SimpleImputer()),
        ('std scale',StandardScaler()),
        ('reg',LinearRegression())
    ]
)


fig,axs=plt.subplots(1,1,figsize=(6,4),sharex=True) 

axs.set_ylabel('RMS Error')
axs.set_xlabel('Training Data Records')
axs.set_title('RMSE versus Training Record Length')


# Xy_val=df_clip_row1.loc[:date_val_end]
Xy_val=df_imputed.loc[:date_val_end]
print(len(Xy_val))
Xy_val=Xy_val.loc[~(Xy_val.index=='2022-12-02')]
print(len(Xy_val))
Xy_val.loc[:,'Desks_Booked']=df_pre_noise['Desks_Booked']

X = Xy_val.drop(columns=[label_field])
y = Xy_val[label_field]


tscv = TimeSeriesSplit(gap=0, max_train_size=None, n_splits=20, test_size=10)
# TimeSeriesSplit(gap=0, max_train_size=30, n_splits=splt, test_size=None)
for label,reg_pipe in [('Linear SVR',SVR_pipe),('Linear Regression',LR_pipe)]:
    rmse=[]
    training_size=[]
    for i, (train_index, test_index) in enumerate(tscv.split(X)):

        X_train,y_train=X.iloc[train_index],y.iloc[train_index]
        X_test,y_test=X.iloc[test_index],y.iloc[test_index]
        
        reg_pipe.fit(X_train,y_train)
        y_pred=reg_pipe.predict(X_test)
        met1=r2_score(y_test,y_pred)
        met2=mean_absolute_error(y_test,y_pred)
        met3=root_mean_squared_error(y_test,y_pred)

        # print(f'Regressor:  R^2 Score {met1:.3f}, Mean Abs Error {met2:.3f}, RMS Error {met3:.3f} Training Size {len(X_train)}')
        rmse.append(met3)
        training_size.append(len(X_train))
    print(training_size)
    print(rmse)
    axs.plot(training_size,rmse,label=label)
axs.grid(visible=True,which='Major',axis='both')
axs.legend()
        

fig.tight_layout()

fig.savefig('./Output Files/Images/Data Exploration/regression_fold_qc.png',format='png',bbox_inches='tight')

_ = plt.show()

In [ ]:
_ = pds.GetScores()

with engine.connect() as conn:
    pds.metricframe.to_sql('PreProc_Metrics',conn,schema='Gold',if_exists='replace',index=False,dtype=sqlcol(pds.metricframe))